# Chemical equilibrium with fixed pH and charge balance

**Recap**: Imposing pH of a solution in a chemical equilibrium calculation leaves the system open to H<sup>+</sup>. This results into the final solution with nonzero electrical charge.

Below, we demonstrate how to perform chemical equilibrium calculations with **pH** and **charge balance** restrictions. The chemical system below is defined with single aqueous phase with thermodynamic data fetched from the [SupcrtDatabase](https://reaktoro.org/api/classReaktoro_1_1SupcrtDatabase.html) object:

In [1]:
from reaktoro import *

db = SupcrtDatabase("supcrtbl")

solution = AqueousPhase("H2O(aq) H+ OH- Na+ Cl- HCO3- CO2(aq) CO3-2")
solution.setActivityModel(ActivityModelDebyeHuckel())

system = ChemicalSystem(db, solution)

Since

* temperature,
* pressure,
* pH, and
* electric charge

must be constrained and system is already open to entering/leaving H<sup>+</sup>, we need to make it open to an anion (negative ion) to counterbalance positive change of hydrogen ion. This is done by Cl<sup>-</sup>:

In [2]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()          # opens the system to H+
specs.charge()
specs.openTo("Cl-") # needs to be set to counterbalance positive charge of H+

solver = EquilibriumSolver(specs)

The *initial chemical state* for our chemical system is defined as follows:

In [3]:
state = ChemicalState(system)
state.temperature(30.0, "celsius")
state.pressure(1.0, "bar")
state.set("H2O(aq)", 1.00, "kg")
state.set("Na+"    , 0.01, "mol")
state.set("Cl-"    , 0.01, "mol")
state.set("CO2(aq)", 0.10, "mol")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    303.1500 |    K |
| Pressure        |      1.0000 |  bar |
| Charge:         | -3.0000e-16 |  mol |
| Element Amount: |             |      |
| :: H            |  1.1102e+02 |  mol |
| :: C            |  1.0000e-01 |  mol |
| :: O            |  5.5708e+01 |  mol |
| :: Na           |  1.0000e-02 |  mol |
| :: Cl           |  1.0000e-02 |  mol |
| Species Amount: |             |      |
| :: H2O(aq)      |  5.5508e+01 |  mol |
| :: H+           |  1.0000e-16 |  mol |
| :: OH-          |  1.0000e-16 |  mol |
| :: Na+          |  1.0000e-02 |  mol |
| :: Cl-          |  1.0000e-02 |  mol |
| :: HCO3-        |  1.0000e-16 |  mol |
| :: CO2(aq)      |  1.0000e-01 |  mol |
| :: CO3-2        |  1.0000e-16 |  mol |
+-----------------+-------------+------+


Next, we set the temperature, pressure, pH, and charge conditions we want to impose at equilibrium for this aqueous solution currently in disequilibrium and equilibrate (using verification of calculation success):

In [4]:
conditions = EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.pH(2.0)
conditions.charge(0.0)

result = solver.solve(state, conditions)
print("Successful computation!" if result.optima.succeeded else "Computation has failed!")

print("FINAL STATE")
print(state)

Successful computation!
FINAL STATE
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    323.1500 |    K |
| Pressure        |     10.0000 |  bar |
| Charge:         | -2.2055e-18 |  mol |
| Element Amount: |             |      |
| :: H            |  1.1103e+02 |  mol |
| :: C            |  1.0000e-01 |  mol |
| :: O            |  5.5708e+01 |  mol |
| :: Na           |  1.0000e-02 |  mol |
| :: Cl           |  2.1127e-02 |  mol |
| Species Amount: |             |      |
| :: H2O(aq)      |  5.5508e+01 |  mol |
| :: H+           |  1.1133e-02 |  mol |
| :: OH-          |  6.1358e-12 |  mol |
| :: Na+          |  1.0000e-02 |  mol |
| :: Cl-          |  2.1127e-02 |  mol |
| :: HCO3-        |  6.1507e-06 |  mol |
| :: CO2(aq)      |  9.9994e-02 |  mol |
| :: CO3-2        |  6.0702e-14 |  mol |
+-----------------+-------------+------+


**TASK**: 1) evaluate and output aqueous properties to make sure that fixed properties are met 2) investigate how much H<sup>+</sup> and Cl<sup>-</sup> entered/leaved the system (compared to the initial state) so that both pH and charge conditions could be reached.

In [5]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #
aprops = AqueousProps(state)

print("AQUEOUS PROPERTIES AT EQUILIBRIUM")
print(aprops)

print("Amount of H+  that entered the system:", state.equilibrium().implicitTitrantAmounts()[0], "mol")
print("Amount of Cl- that entered the system:", state.equilibrium().explicitTitrantAmounts()[0], "mol")

AQUEOUS PROPERTIES AT EQUILIBRIUM
+-----------------------------------+------------+-------+
| Property                          |      Value |  Unit |
+-----------------------------------+------------+-------+
| Temperature                       |   323.1500 |     K |
| Pressure                          |    10.0000 |   bar |
| Ionic Strength (Effective)        |     0.0211 | molal |
| Ionic Strength (Stoichiometric)   |     0.0211 | molal |
| pH                                |     2.0000 |       |
| pE                                |     1.3421 |       |
| Eh                                |     0.0861 |     V |
| Element Molality:                 |            |       |
| :: C                              | 1.0000e-01 | molal |
| :: Na                             | 1.0000e-02 | molal |
| :: Cl                             | 2.1127e-02 | molal |
| Species Molality:                 |            |       |
| :: H+                             | 1.1133e-02 | molal |
| :: OH-              

We can now see that pH is exactly 2, and temperature and pressure are 50 °C and 10 bar (but displayed in the table as 323.15 K and 10<sup>6</sup> Pa respectively). The results shown above demonstrate that our constraints were satisfied at equilibrium and that both H<sup>+</sup> and Cl<sup>-</sup> entered the system with a numerically equal amount.